In [1]:
# !/usr/bin/python
# -*- coding: utf-8 -*-
# Author: Selvaria
# 按照北京设计的数据库结构获取企业深度信息（股东，财务信息，关系图等）,并存储
# 目前算上登录，每次爬取需向网站发起约10次访问，企查查24小时内访问量超过3万次封禁账号一周

import requests
import time
from bs4 import BeautifulSoup
import csv
import pandas
import random
from selenium import webdriver
import json
import pymysql
import json
import concurrent.futures
from DBUtils.PooledDB import PooledDB
import logging
import datetime

now = datetime.datetime.now()
today = now.strftime('%Y-%m-%d')
LOG_FORMAT = "%(asctime)s - %(levelname)s - %(message)s"
DATE_FORMAT = "%m/%d/%Y %H:%M:%S %p"

logging.basicConfig(filename='./Logs/'+today+'qichacha.log', level=logging.INFO, format=LOG_FORMAT, datefmt= DATE_FORMAT)


_dbservice_pool = None

class MysqlPool(object): #设置数据库连接池和初始化    
    """
        MYSQL数据库对象，负责产生数据库连接 , 此类中的连接采用连接池实现
        获取连接对象：conn = Mysql.get_connection()
        释放连接对象;conn.close()或del conn
    """
    
    def __init__(self,mincached=5, maxcached=14,
                maxconnections=244, blocking=True, maxshared=0):
        """
            生成MySQL数据库连接池
        :param mincached: 最少的空闲连接数，如果空闲连接数小于这个数，pool会创建一个新的连接
        :param maxcached: 最大的空闲连接数，如果空闲连接数大于这个数，pool会关闭空闲连接
        :param maxconnections: 最大的连接数
        :param blocking: 当连接数达到最大的连接数时，在请求连接的时候，如果这个值是True，请求连接的程序会一直等待，
                          直到当前连接数小于最大连接数，如果这个值是False，会报错，
        :param maxshared: 当连接数达到这个数，新请求的连接会分享已经分配出去的连接
        """        
        db_config = {
            "host": '47.92.25.70',
            "port": 3306,
            "user": 'root',
            "passwd": 'Wfn031641',
            "db": 'zhaoshang',
            "charset": 'utf8'
        }
        self.pool = PooledDB(pymysql, mincached=mincached, maxcached=maxcached, maxconnections=maxconnections,
                             blocking=blocking, maxshared=maxshared, **db_config)

    def get_connection(self):
        return self.pool.connection()

    def close(self):
        self.pool.close()

    def __del__(self):
        self.close()

# 获取mysql数据库服务器的连接
def get_dbservice_mysql_conn(): #实例化连接池类，方便调用
    """
    :return: Object  MySQL Connection
    """
    global _dbservice_pool
    if not _dbservice_pool or not isinstance(_dbservice_pool, MysqlPool):
        _dbservice_pool = MysqlPool()
    return _dbservice_pool.get_connection()



class QichachaInfo(object):
    
    headers = {
                'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.80 Safari/537.36'
            }
    #browser = webdriver.Chrome()
    
    '''def __init__(self, name):
        self.name = name'''
    
    def get_login(self): #利用selenium手动登录,获取cookies
        url = 'https://www.qichacha.com/user_login'
        browser = webdriver.Chrome() 
        browser.get(url) 
        #login_click = browser.find_element_by_xpath('/html/body/header/div/ul[2]/li[7]/a')  #登陆
        #browser.execute_script("arguments[0].click();", login_click)
        time.sleep(30) #登录并录入验证码  #15201760096 123456  #18698925437 #13342264161 
        #req = requests.Session()
        cookies = browser.get_cookies()
        post={} #定义一个空的字典，存放cookies内容
        #获取到的cookies是列表形式，将cookies转成字典格式
        for cookie_item in cookies:
            post[cookie_item['name']] = cookie_item['value'] 
        browser.quit()
        cookie_str = json.dumps(post)
        return post,cookies

    def get_company_code(self, cookies, name): #获取企业代号No（企查查）
        #cookies = self.get_login()
        #req = requests.Session()
        #for cookie in cookies:
            #req.cookies.set(cookie['name'],cookie['value'])
        #print(cookies)
        company_name = name
        #browser = webdriver.Chrome() 
        #browser.get(url)

        #browser.get('https://www.qichacha.com/search?key=%s' %company_name)

        url_s = requests.get('https://www.qichacha.com/search?key=%s' %company_name, headers = self.headers, cookies = cookies).content
        soup_s = BeautifulSoup(url_s,'html.parser')
        try:
            num_u = soup_s.find('span', attrs = {'id': "countOld"})
            num = num_u.find('span', attrs = {'class': "text-danger"})
        except Exception as e:
            logging.warning('基本被封，请手动查询')
            print(str(e))
            return None
            #self.get_login()
            exit(1) # 有错误退出

        if num and num.get_text().strip() == '0':
            print('无此企业', name)
            return 0
        else:
            time.sleep(1)
            search_url = soup_s.find('a', attrs = {'class': "ma_h1"})['href']
            company_code = search_url[6:-5]
            #url_company = requests.get('https://www.qichacha.com%s' %search_url, headers = self.headers).content
            com_web_q = 'https://www.qichacha.com' + search_url #企查查的企业主页
            return company_code
        
    
class StockMember(QichachaInfo): #股权穿透信息, 2次申请访问
    
    headers = {
                'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.80 Safari/537.36'
            }
    
    def __init__(self, cookies):
        self.cookies = cookies
    
    def get_stock_info(self, code): #对应code公司的股份原始数据
        url_head = 'https://www.qichacha.com/cms_guquanmap3'
        params = {'keyNo':code}
        data_ori = requests.get(url_head, params = params, headers = self.headers, cookies= self.cookies).json()
        #print(data_ori)
        return data_ori    
    
    def get_gudong_list(self, code, data_json): #获取股东多级列表
        #data_json = self.get_stock_info(code)
        #print(data_json)
        gudongs = data_json['gudong']['DetailList']
        gudong_list=[]
        for dict_i in gudongs:
            name_p = dict_i['Name']
            name_id = dict_i['KeyNo']
            percent = dict_i['PercentTotal']
            amount = dict_i['StockRightNum'] #持股数
            data_row = [code, name_p, name_id, percent, amount]
            gudong_list.append(data_row)
            if len(dict_i['DetailList'])>0:
                self.up_sql_id(dict_i['Name'],dict_i['KeyNo'])
                data_sub = dict_i['DetailList']
                for dict_sub in data_sub:
                    name_p = dict_sub['Name']
                    name_id_sub = dict_sub['KeyNo']
                    percent = dict_sub['Percent'] #再下级股东比例，相加应为100%
                    amount = dict_sub['StockRightNum'] 
                    data_row = [name_id, name_p, name_id_sub, percent, amount]
                    gudong_list.append(data_row)
        return gudong_list
    
    def get_kzr(self, code,data_json): #获取实际控制人
        #data_json = self.get_stock_info(code)
        kzr_data = data_json['kzr']
        company_id = kzr_data['KeyNo']
        kzr_name = kzr_data['ControllerData']['Name']
        kzr_id = kzr_data['ControllerData']['KeyNo']
        kzr_precent = kzr_data['ControllerData']['PercentTotal']
        data_row = [company_id, kzr_name, kzr_id, kzr_precent]
        return data_row
    
    def get_touzi_list(self, code,data_json): #获取公司投资列表
        #data_json = self.get_stock_info(code)
        touzis = data_json['touzi']['DetailList']
        touzi_list=[]
        for dict_i in touzis:
            name_p = dict_i['Name']
            name_id = dict_i['KeyNo']
            percent = dict_i['PercentTotal']
            amount = dict_i['ShouldCapi'] #认缴金额（万）
            data_row = [code, name_p, name_id, percent, amount]
            touzi_list.append(data_row)
            if len(dict_i['DetailList'])>0:
                self.up_sql_id(dict_i['Name'],dict_i['KeyNo'])
                data_sub = dict_i['DetailList']
                for dict_sub in data_sub:
                    name_p = dict_sub['Name']
                    name_id_sub = dict_sub['KeyNo']
                    percent = dict_sub['Percent'] #再向下投资的比例
                    amount = dict_sub['ShouldCapi'] 
                    data_row = [name_id, name_p, name_id_sub, percent, amount]
                    touzi_list.append(data_row)
        return touzi_list
    
    def up_sql_id(self, name, code): #单独更新企业id对应表
        connection = get_dbservice_mysql_conn()
        try :
            with connection.cursor() as cursor:
                sql_id = "UPDATE `dic_company` SET `q_id`= %s WHERE `com` = %s"
                cursor.execute(sql_id,(code,name))
            connection.commit()
            #print(code,name)
        except Exception as e:
            print('%s企业q_id数据写入失败' %name)
            print(str(e))
    
    def up_sql_guquan(self, name, code):
        #q_info = [code,name]
        self.up_sql_id(name, code)
        data_json = self.get_stock_info(code)
        gudong_list = self.get_gudong_list(code, data_json)
        try:
            kzr = self.get_kzr(code, data_json)
        except:
            print('%s企业无控制人信息' %name)
            kzr = [code,]+['']*3
        try:
            touzi_list = self.get_touzi_list(code, data_json)
        except:
            print('%s企业无投资信息' %name)
            touzi_list = [code,]+['']*4
        connection = get_dbservice_mysql_conn()
        #print(touzi_list)
        try :
            for gudong_data in gudong_list:
                with connection.cursor() as cursor:
                    sql_g = "insert into `com_partner` (`com_id`,`person`,`partner_id`,`percent`,`amount`)\
                    values(%s,%s,%s,%s,%s)"
                    cursor.execute(sql_g,gudong_data)
                connection.commit()
            print('gudong partner data is done!')
        except Exception as e:
            print('%s企业股权数据写入失败' %name)
            print(str(e))
        try:
            for gudong_data in gudong_list:
                try:
                    with connection.cursor() as cursor:
                        sql = "insert into `dic_person` (`name`,`q_id`) values(%s,%s)"
                        cursor.execute(sql,gudong_data[1:3])
                    connection.commit()
                except pymysql.err.IntegrityError:
                    continue
            print('gudong person data is done!')
        except Exception as e:
            print('%s股东单独表数据写入失败' %name)
            print(str(e))
        
        try :
            for touzi_data in touzi_list:
                with connection.cursor() as cursor:
                    sql_t = "insert into `com_invest` (`invest_com_id`,`com_name`,`com_id`,`percent`,`amount`)\
                    values(%s,%s,%s,%s,%s)"
                    cursor.execute(sql_t,touzi_data)
                connection.commit()
            print('touzi data is done!')
        except Exception as e:
            print('%s企业投资数据写入失败' %name)
            print(str(e))
        try :
            with connection.cursor() as cursor:
                sql_k = "insert into `com_kzr` (`q_id`,`name`,`name_id`,`percent`) values(%s,%s,%s,%s)"
                cursor.execute(sql_k,kzr)
            connection.commit()
        except Exception as e:
            print('%s企业实际控制人数据写入失败' %name)
            print(str(e))
        finally:
            connection.close()
            #print('%s guquan info is done!' %name)
            
    def up_sql_g_form(self,code): #上传整个图的json数据
        data_json = self.get_stock_info(code)
        data_sql = json.dumps(data_json)
        connection = get_dbservice_mysql_conn()
        try :
            with connection.cursor() as cursor:
                sql = "insert into `com_atlas` (`com_id`,`atlas_type`,`atlas_data`) values(%s,%s,%s)"
                cursor.execute(sql,(code,'股权结构图',data_sql))
            connection.commit()
            print('guquan_touzi form is Done!')
        except Exception as e:
            print('%s企业图数据写入失败' %code)
            print(str(e))
    
    
class FinancialStatus(QichachaInfo): #财务年度数据, 1次申请访问
    
    def __init__(self, cookies):
        self.cookies = cookies
    
    def get_financial_info(self, code):
        url_head = 'https://www.qichacha.com/company_financialanalysis'
        #req = requests.Session()
        #for cookie in cookies:
            #req.cookies.set(cookie['name'],cookie['value'])
        params = {'keyno':code}
        data_ori = requests.get(url_head, params = params, headers = self.headers, cookies= self.cookies).json()
        if data_ori['Status'] == 201:
            print('%s未公布财务数据' %code)
            return None
        else:
            return data_ori['Result']
        
    def get_gross_profit(self, code,data_info):  #年毛利率
        #data_info = self.get_financial_info(code)
        if data_info is None:
            return None
        gross_profit = data_info['GrossProfitRateList']
        data_list =[]
        for data in gross_profit['DataList']:
            value = data['Value']
            year = data['Year']
            data_list.append([code, value, year])
        return data_list
    
    def get_clean_asset(self, code, data_info):  #年净资产
        #data_info = self.get_financial_info(code)
        if data_info is None:
            return None
        clean_asset = data_info['NetAssetsList']
        data_list =[]
        for data in clean_asset['DataList']:
            value = data['Value']
            year = data['Year']
            data_list.append([code, value, year])
        return data_list
    
    def get_clean_margin(self, code, data_info):  #年净利润
        #data_info = self.get_financial_info(code)
        if data_info is None:
            return None
        clean_margin = data_info['NetMarginList']
        data_list =[]
        for data in clean_margin['DataList']:
            value = data['Value']
            year = data['Year']
            data_list.append([code, value, year])
        return data_list
    
    def get_clean_profit_rate(self, code, data_info):  #年净利率
        #data_info = self.get_financial_info(code)
        if data_info is None:
            return None
        clean_profit_rate = data_info['NetProfitRateList']
        data_list =[]
        for data in clean_profit_rate['DataList']:
            value = data['Value']
            year = data['Year']
            data_list.append([code, value, year])
        return data_list
    
    def get_revenue(self, code, data_info):  #年营业收入
        #data_info = self.get_financial_info(code)
        if data_info is None:
            return None
        revenue = data_info['OperationRevenueList']
        data_list =[]
        for data in revenue['DataList']:
            value = data['Value']
            year = data['Year']
            data_list.append([code, value, year])
        return data_list
    
    def get_total_asset(self, code, data_info):  #总资产
        #data_info = self.get_financial_info(code)
        if data_info is None:
            return None
        total_asset = data_info['TotalAssetsList']
        data_list =[]
        for data in total_asset['DataList']:
            value = data['Value']
            year = data['Year']
            data_list.append([code, value, year])
        return data_list
    
    def up_sql_financial(self, code):
        data_info = self.get_financial_info(code)
        gross_profit = self.get_gross_profit(code,data_info)
        clean_asset = self.get_clean_asset(code,data_info)
        clean_margin = self.get_clean_margin(code,data_info)
        clean_profit_rate = self.get_clean_profit_rate(code,data_info)
        revenue = self.get_revenue(code,data_info)
        total_asset = self.get_total_asset(code,data_info)
        connection = get_dbservice_mysql_conn()
        try :
            for data in gross_profit:
                with connection.cursor() as cursor:
                    sql_f = "insert into `com_f_gross_profit` (`q_id`,`value`,`year`) values(%s,%s,%s)"
                    cursor.execute(sql_f,data)
                connection.commit()
            for data in clean_asset:
                with connection.cursor() as cursor:
                    sql_f = "insert into `com_f_clean_asset` (`q_id`,`value`,`year`) values(%s,%s,%s)"
                    cursor.execute(sql_f,data)
                connection.commit()
            for data in clean_margin:
                with connection.cursor() as cursor:
                    sql_f = "insert into `com_f_clean_margin` (`q_id`,`value`,`year`) values(%s,%s,%s)"
                    cursor.execute(sql_f,data)
                connection.commit()
            for data in clean_profit_rate:
                with connection.cursor() as cursor:
                    sql_f = "insert into `com_f_clean_profit_rate` (`q_id`,`value`,`year`) values(%s,%s,%s)"
                    cursor.execute(sql_f,data)
                connection.commit()
            for data in revenue:
                with connection.cursor() as cursor:
                    sql_f = "insert into `com_f_revenue` (`q_id`,`value`,`year`) values(%s,%s,%s)"
                    cursor.execute(sql_f,data)
                connection.commit()
            for data in total_asset:
                with connection.cursor() as cursor:
                    sql_f = "insert into `com_f_total_asset`(`q_id`,`value`,`year`) values(%s,%s,%s)"
                    cursor.execute(sql_f,data)
                connection.commit()
            print('%s financial info is done!' %code) 
        except Exception as e:
            print('%s企业财务数据写入失败' %code)
            print(str(e))
        finally:
            connection.close()
            
            
class CompanyConnect(QichachaInfo):  #企业关联图谱, 1或2次申请访问
    
    def __init__(self, cookies):
        self.cookies = cookies
        
    def get_connection_info(self, code):
        url_head = 'https://www.qichacha.com/company_muhouInsertIndex'
        #req = requests.Session()
        #for cookie in cookies:
            #req.cookies.set(cookie['name'],cookie['value'])
        params = {'keyNo':code}
        data_ori = requests.get(url_head, params = params, headers = self.headers, cookies= self.cookies).json()
        data_ana = data_ori['success']['results'][0]['data'][0]['graph']
        return data_ana
    
    def get_nodes_info(self, code, data_info): #节点信息
        #data_info = self.get_connection_info(code)['nodes']
        data_row = []
        for node_dict in data_info:
            node_id = node_dict['id']
            label = node_dict['labels'][0]
            q_id = node_dict['properties']['keyNo']
            name = node_dict['properties']['name']
            try:
                role = node_dict['properties']['role']
            except:
                role = node_dict['properties']['econKind']
            data_node = [code, node_id, label, q_id, name, role]
            data_row.append(data_node)
        return data_row
    
    def get_links_info(self, code, data_info): #节点关联信息
        #data_info = self.get_connection_info(code)['relationships']
        data_row = []
        for link_dict in data_info:
            link_id = link_dict['id']
            start = link_dict['startNode']
            end = link_dict['endNode']
            type_link = link_dict['type']
            #percent = link_dict['properties']['stockPercent']
            try:
                role = link_dict['properties']['role']                
            except:
                role = ''
            try:
                percent = link_dict['properties']['stockPercent']                
            except:
                percent = ''
            data_link = [code, link_id, start, end, type_link, role, percent]
            data_row.append(data_link)
        return data_row
    
    def up_sql_connect(self,code):
        data_info = self.get_connection_info(code)['nodes']
        node_list = self.get_nodes_info(code,data_info)
        link_list = self.get_links_info(code,data_info)
        #print(touzi_list)
        connection = get_dbservice_mysql_conn()
        try :
            for node_data in node_list:
                with connection.cursor() as cursor:
                    sql_n = "insert into `connect_nodes` (`q_id`,`node_id`,`label`,`ref_id`,`name`,`role`)\
                    values(%s,%s,%s,%s,%s,%s)"
                    cursor.execute(sql_n,node_data)
                connection.commit()
        except Exception as e:
            print('%s企业节点数据写入失败' %code)
            print(str(e))
        try :
            for link_data in link_list:
                with connection.cursor() as cursor:
                    sql_l = "insert into `connect_links` (`q_id`,`link_id`,`start`,`end`,`type`,`role`,`percent`)\
                    values(%s,%s,%s,%s,%s,%s,%s)"
                    cursor.execute(sql_l,link_data)
                connection.commit()
            print('%s node-link table is done!' %code)
        except Exception as e:
            print('%s企业连接数据写入失败' %code)
            print(str(e))
        finally:
            connection.close()
            #print('%s node-link table is done!' %code)
            
    def up_sql_c_form(self,code): #上传整个图的json数据
        data_json = self.get_connection_info(code)
        data_sql = json.dumps(data_json)
        connection = get_dbservice_mysql_conn()
        try :
            with connection.cursor() as cursor:
                sql = "insert into `com_atlas` (`com_id`,`atlas_type`,`atlas_data`) values(%s,%s,%s)"
                cursor.execute(sql,(code,'企业关系图',data_sql))
            connection.commit()
            print('%s connect link is done!' %code)
        except pymysql.err.IntegrityError:
            pass
        except Exception as e:
            print('%s企业关联图数据写入失败' %code)
            print(str(e))
        finally:
            connection.close()
            
            
class ZhuanLi(QichachaInfo): #专利信息，1次申请访问
    
    def __init__(self, cookies):
        self.cookies = cookies
        
    def get_zhuanli_data(self, code):
        url = 'https://www.qichacha.com/cassets_' + code
        CHROME_OPTIONS = webdriver.ChromeOptions()
        CHROME_OPTIONS.add_argument('headless')
        driver = webdriver.Chrome(chrome_options=CHROME_OPTIONS)
        #driver = webdriver.Chrome()
        url_l= 'https://www.qichacha.com/user_login'
        driver.get(url_l) 
        for cookie_item in self.cookies:
            #print({'name': cookie_item, 'value': cookies[cookie_item]})
            driver.add_cookie(cookie_item)
        driver.get(url)
        time.sleep(4)
        count = driver.find_element_by_xpath('//*[@id="assets_title"]/span')
        if count.text == '0':
            print('%s无知识产权' %code)
            return None
        zhuanli = driver.find_element_by_id("zhuanlilist")
        table = driver.find_element_by_xpath('//*[@id="zhuanlilist"]/table/tbody').text
        table_data = self.zl_table_handle(table)
        try:
            page_num = zhuanli.find_elements_by_id("ajaxpage")
            page_max = int(page_num[-1].text[-1])
            for i in range(page_max-1):
                zhuanli = driver.find_element_by_id("zhuanlilist")
                next_page = zhuanli.find_elements_by_id("ajaxpage")[-2]
                if '>' in next_page.text:
                    pass
                else:
                    next_page = zhuanli.find_elements_by_id("ajaxpage")[-1]
                driver.execute_script("arguments[0].click();",next_page)
                time.sleep(2)
                table = driver.find_element_by_xpath('//*[@id="zhuanlilist"]/table/tbody').text
                table_data.extend(self.zl_table_handle(table))
            return table_data
        except Exception as e:
            print(str(e))
            return table_data
    
    def zl_table_handle(self,table): #原始数据处理
        table_row = table.split('\n')
        row_list =[]
        for row in table_row:
            row_s = row.split()
            row_list.append(row_s)
        row_list.pop(0)
        return row_list
    
    def up_sql_zhuanli(self, code):
        data = self.get_zhuanli_data(code)
        if data == None:
            return False
        data_sql = [row+[code,] for row in data]
        connection = get_dbservice_mysql_conn()
        for z_data in data_sql:
            try :
                with connection.cursor() as cursor:
                    sql_z = "insert into `com_patent` (`index`,`type`,`patent_num`,`apply_date`,`patent_name`,`com_id`)\
                    values(%s,%s,%s,%s,%s,%s)"
                    cursor.execute(sql_z,z_data)
                connection.commit()
            except Exception as e:
                print('%s专利数据写入失败' %z_data)
                print(str(e))
                continue
        connection.close()
        print('%s zhuanli info is done!' %code)
        

class ChangeRecord(QichachaInfo): #公司变动信息,主要人员信息,及补充的基本信息，2次申请访问
    
    def __init__(self, cookies):
        self.cookies = cookies
        
    '''url_l= 'https://www.qichacha.com/user_login'
    CHROME_OPTIONS = webdriver.ChromeOptions()
    CHROME_OPTIONS.add_argument('headless')
    browser = webdriver.Chrome(chrome_options=CHROME_OPTIONS)
    browser.get(url_l) 
    for cookie_item in self.cookies:
        browser.add_cookie(cookie_item)'''
        
        
    def get_change_data(self, code): #公司变动信息,主要人员信息
        url_t = 'https://www.qichacha.com/cbase_' + code
        url_l= 'https://www.qichacha.com/user_login'
        CHROME_OPTIONS = webdriver.ChromeOptions()
        CHROME_OPTIONS.add_argument('headless')
        browser = webdriver.Chrome(chrome_options=CHROME_OPTIONS)
        browser.get(url_l) 
        for cookie_item in self.cookies:
            browser.add_cookie(cookie_item)
        #browser = webdriver.Chrome() 
        browser.get(url_t)
        time.sleep(4)
        try:
            table = browser.find_element_by_xpath('//*[@id="Changelist"]/table') #变动信息表
            table_rows = table.find_elements_by_tag_name('tr')
            table_list = []
            for row in table_rows:
                table_list.append(row.find_elements_by_tag_name('td'))
            decode_list =[]
            for row in table_list:
                cell_data = [cell.text.replace('\n','；') for cell in row]
                decode_list.append(cell_data)
            decode_list.pop(0)
        except:
            decode_list = None
        #time.sleep(2)
        try:
            table_partner = browser.find_element_by_xpath('//*[@id="Mainmember"]/table') #主要人员表
            table_rows = table_partner.find_elements_by_tag_name('tr')
            table_list = []
            for row in table_rows:
                table_list.append(row.find_elements_by_tag_name('td'))
            partner_list =[]
            for row in table_list:
                cell_data = [cell.text.replace('\n','；') for cell in row]
                partner_list.append(cell_data)
            partner_list.pop(0)
            for i in partner_list:
                name_index =i[1].index('；')
                name = i[1][:name_index]
                i.pop(1)
                i.insert(1,name)
        except Exception as e:
            partner_list = None
            print(str(e))
        return decode_list, partner_list    
    
    def up_sql_change(self, code):
        data = self.get_change_data(code)
        data_change = data[0]
        data_partner = data[1]
        if data_change == None:
            return False
        data_sql_c = [row+[code,] for row in data_change]
        data_sql_p = [row+[code,] for row in data_partner]
        connection = get_dbservice_mysql_conn()
        for c_data in data_sql_c:
            #print(c_data)
            try :
                with connection.cursor() as cursor:
                    sql_c = "insert into `com_change_detail` (`index`,`time`,`name`,`before`,`after`,`q_id`)\
                    values(%s,%s,%s,%s,%s,%s)"
                    cursor.execute(sql_c,c_data)
                connection.commit()
            except Exception as e:
                print('%s变动数据写入失败' %code)
                print(str(e))
                continue
        #connection.close()
        print('%s change info is done!' %code)
        for p_data in data_sql_p:
            #print(p_data)
            try :
                with connection.cursor() as cursor:
                    sql_p = "insert into `com_members` (`index`,`name`,`charactor`,`q_id`)\
                    values(%s,%s,%s,%s)"
                    cursor.execute(sql_p,p_data)
                connection.commit()
            except Exception as e:
                print('%s主要人员数据写入失败' %code)
                print(str(e))
                continue
        connection.close()
        print('%s partner info is done!' %code)
        
    def get_com_status(self,code): #公司经营状态信息
        url_t = 'https://www.qichacha.com/cbase_' + code
        url_l= 'https://www.qichacha.com/user_login'
        CHROME_OPTIONS = webdriver.ChromeOptions()
        CHROME_OPTIONS.add_argument('headless')
        browser = webdriver.Chrome(chrome_options=CHROME_OPTIONS)
        #browser = webdriver.Chrome() 
        browser.get(url_l) 
        for cookie_item in self.cookies:
            browser.add_cookie(cookie_item)
        browser.get(url_t)
        time.sleep(4)
        info = browser.find_element_by_xpath('//*[@id="Cominfo"]/table[2]/tbody/tr[2]/td[2]').text
        if '开业' in info:
            info = '在业'
        elif '在业' in info:
            info = '在业'
        elif '注销' in info:
            info = '注销'
        connection = get_dbservice_mysql_conn()
        with connection.cursor() as cursor:
            sql = "select `id` from `dic_com_status` where `com_status` = %s"
            cursor.execute(sql,(info,))
            info_id  = cursor.fetchone()
        status_id = info_id[0]
        return status_id
    
    def update_status(self, code):
        status_id = self.get_com_status(code)
        connection = get_dbservice_mysql_conn()
        with connection.cursor() as cursor:
            sql_v = "update `dic_company` set `com_status` = %s where `q_id` = %s"
            cursor.execute(sql_v,(status_id,code))
        connection.commit()
        print('%s营业状态修改完成' %code)
        

class GoldMerge(QichachaInfo): #融资信息，1次申请访问
    
    def __init__(self, cookies):
        self.cookies = cookies
        
    def get_merge_data(self, code):
        url_t = 'https://www.qichacha.com/crun_' + code
        url_l= 'https://www.qichacha.com/user_login'
        CHROME_OPTIONS = webdriver.ChromeOptions()
        CHROME_OPTIONS.add_argument('headless')
        browser = webdriver.Chrome(chrome_options=CHROME_OPTIONS)
        #browser = webdriver.Chrome() 
        browser.get(url_l) 
        for cookie_item in self.cookies:
            browser.add_cookie(cookie_item)
        browser.get(url_t)
        time.sleep(4)
        table = browser.find_element_by_xpath('//*[@id="financingList"]/table')
        table_rows = table.find_elements_by_tag_name('tr')
        table_list = []
        for row in table_rows:
            table_list.append(row.find_elements_by_tag_name('td'))
        decode_list =[]
        for row in table_list:
            cell_data = [cell.text.replace('\n','；') for cell in row]
            decode_list.append(cell_data)
        decode_list.pop(0)
        return decode_list    
    
    def up_sql_merge(self, code):
        data = self.get_merge_data(code)
        data_sql = [row+[code,] for row in data]
        connection = get_dbservice_mysql_conn()
        for c_data in data_sql:
            with connection.cursor() as cursor:
                sql = "select `id` from `dic_financing_type` where `financing_type` = %s"
                cursor.execute(sql,(c_data[3],))
                level_id  = cursor.fetchone()[0]
            c_data.insert(3,level_id)
            try :
                with connection.cursor() as cursor:
                    sql_c = "insert into `merge_detail` (`index`,`financing_date`,`product_name`,`financing_type_id`,`financing_level`,\
                    `money`,`invest_com_name`,`com_id`)\
                    values(%s,%s,%s,%s,%s,%s,%s,%s)"
                    cursor.execute(sql_c,c_data)
                connection.commit()
            except Exception as e:
                print('%s融资数据写入失败' %code)
                print(str(e))
                continue
        connection.close()
        print('%s merge info is done!' %code)



In [4]:
def financial_sql(): #单独更新财务信息
    q = QichachaInfo()
    cookies_tuple = q.get_login()
    cookies = cookies_tuple[0]
    cookies_q = cookies_tuple[1]
    
    f = FinancialStatus(cookies)
    s = StockMember(cookies)
    c = CompanyConnect(cookies)
    connection = get_dbservice_mysql_conn()
    sql = "select `id`,`com`,`q_id` from `dic_company`"
    df = pandas.read_sql(sql,connection)
    count = 0
    for index in range(2774, 98921):
        print('开始进行企业index:',index)
        name = df.loc[index, 'com']
        #code = df.loc[index, 'q_id']
        count+=1
        print('Now count:',count)
        if count == 200:
            print('已到限值，请休息')
            time.sleep(840)
            count = 0
            #break
        #else:
        code = df.loc[index, 'q_id']
        try:
            f.up_sql_financial(code)    #财务信息
        except:
            print('%s无公司财务信息,%s' %(name,code))
            print(str(e))
        try:
            c.up_sql_c_form(code)  #企业关联全图json
        except Exception as e:
            print('%s无公司企业关联信息,%s' %(name,code))
            print(str(e))
        try:
            s.up_sql_guquan(name,code) #股权投资详细图
            s.up_sql_g_form(code)      #股权投资全图json
        except Exception as e:
            print('%s无公司股权及投资表信息,%s' %(name,code))
            print(str(e))
        time.sleep(random.uniform(7.4,10.7))
            
financial_sql()

开始进行企业index: 2175
Now count: 1
4fd75eb36aad239d7dc477a16b3b7aae未公布财务数据
4fd75eb36aad239d7dc477a16b3b7aae企业财务数据写入失败
'NoneType' object is not iterable
4fd75eb36aad239d7dc477a16b3b7aae connect link is done!
touzi data is done!
guquan_touzi form is Done!
开始进行企业index: 2176
Now count: 2
20c12c5968534e59fdbef5ff5f33113d未公布财务数据
20c12c5968534e59fdbef5ff5f33113d企业财务数据写入失败
'NoneType' object is not iterable
20c12c5968534e59fdbef5ff5f33113d connect link is done!
上海扬鹏贸易有限公司企业无投资信息
上海扬鹏贸易有限公司企业投资数据写入失败
not enough arguments for format string
guquan_touzi form is Done!
开始进行企业index: 2177
Now count: 3
5ddf9a177668026c09bd2b5ce8fb32f1未公布财务数据
5ddf9a177668026c09bd2b5ce8fb32f1企业财务数据写入失败
'NoneType' object is not iterable
5ddf9a177668026c09bd2b5ce8fb32f1 connect link is done!
上海肯德基有限公司石化餐厅无公司股权及投资表信息,5ddf9a177668026c09bd2b5ce8fb32f1
'NoneType' object is not subscriptable
开始进行企业index: 2178
Now count: 4
1a235c8bda3ce94a0f1d9a1d3d92d7d7未公布财务数据
1a235c8bda3ce94a0f1d9a1d3d92d7d7企业财务数据写入失败
'NoneType' object is not ite

guquan_touzi form is Done!
开始进行企业index: 2203
Now count: 29
b6eb05c4167939c928a7e178d33e702c未公布财务数据
b6eb05c4167939c928a7e178d33e702c企业财务数据写入失败
'NoneType' object is not iterable
b6eb05c4167939c928a7e178d33e702c connect link is done!
上海隆纪石油化工有限公司企业无投资信息
上海隆纪石油化工有限公司企业投资数据写入失败
not enough arguments for format string
guquan_touzi form is Done!
开始进行企业index: 2204
Now count: 30
f01c0ece33905c1ffc2b620610f4e015未公布财务数据
f01c0ece33905c1ffc2b620610f4e015企业财务数据写入失败
'NoneType' object is not iterable
f01c0ece33905c1ffc2b620610f4e015 connect link is done!
上海鸣权石化填料设备有限公司企业无投资信息
上海鸣权石化填料设备有限公司企业投资数据写入失败
not enough arguments for format string
guquan_touzi form is Done!
开始进行企业index: 2205
Now count: 31
769a76c351f02a1ba8c034dc2f6c02b2未公布财务数据
769a76c351f02a1ba8c034dc2f6c02b2企业财务数据写入失败
'NoneType' object is not iterable
769a76c351f02a1ba8c034dc2f6c02b2 connect link is done!
上海崇然石油化工有限公司企业无投资信息
上海崇然石油化工有限公司企业投资数据写入失败
not enough arguments for format string
guquan_touzi form is Done!
开始进行企业index: 2206
Now count: 3

412838b626c34c6d316504a83a0798a2未公布财务数据
412838b626c34c6d316504a83a0798a2企业财务数据写入失败
'NoneType' object is not iterable
412838b626c34c6d316504a83a0798a2 connect link is done!
上海沪彦石油化工有限公司企业无投资信息
上海沪彦石油化工有限公司企业投资数据写入失败
not enough arguments for format string
guquan_touzi form is Done!
开始进行企业index: 2230
Now count: 56
5e3778d61ca45f7655c6bb43f6aeb89c未公布财务数据
5e3778d61ca45f7655c6bb43f6aeb89c企业财务数据写入失败
'NoneType' object is not iterable
5e3778d61ca45f7655c6bb43f6aeb89c connect link is done!
上海兴佳石油化工有限公司企业无投资信息
上海兴佳石油化工有限公司企业投资数据写入失败
not enough arguments for format string
guquan_touzi form is Done!
开始进行企业index: 2231
Now count: 57
0758728cb3706a6e3fd3c1fc92c27d94未公布财务数据
0758728cb3706a6e3fd3c1fc92c27d94企业财务数据写入失败
'NoneType' object is not iterable
0758728cb3706a6e3fd3c1fc92c27d94 connect link is done!
上海嘉宁石油化工有限公司企业无投资信息
上海嘉宁石油化工有限公司企业投资数据写入失败
not enough arguments for format string
guquan_touzi form is Done!
开始进行企业index: 2232
Now count: 58
7bf051a5428ba68a6c0f840750271fba未公布财务数据
7bf051a5428ba68a6c0f8

上海铉熠石油化工有限公司企业无投资信息
上海铉熠石油化工有限公司企业投资数据写入失败
not enough arguments for format string
guquan_touzi form is Done!
开始进行企业index: 2256
Now count: 82
e6d2d55a9d04e01cb283a156be8c3076未公布财务数据
e6d2d55a9d04e01cb283a156be8c3076企业财务数据写入失败
'NoneType' object is not iterable
e6d2d55a9d04e01cb283a156be8c3076 connect link is done!
touzi data is done!
guquan_touzi form is Done!
开始进行企业index: 2257
Now count: 83
f3c0bead7d06aaf7eb123fcfc769928c未公布财务数据
f3c0bead7d06aaf7eb123fcfc769928c企业财务数据写入失败
'NoneType' object is not iterable
f3c0bead7d06aaf7eb123fcfc769928c connect link is done!
上海特雷德石油化工有限公司企业无投资信息
上海特雷德石油化工有限公司企业投资数据写入失败
not enough arguments for format string
guquan_touzi form is Done!
开始进行企业index: 2258
Now count: 84
41191aed6f3d306b5a0ef82e5dc9c0ea未公布财务数据
41191aed6f3d306b5a0ef82e5dc9c0ea企业财务数据写入失败
'NoneType' object is not iterable
41191aed6f3d306b5a0ef82e5dc9c0ea connect link is done!
上海福盟石油化工有限公司企业无投资信息
上海福盟石油化工有限公司企业投资数据写入失败
not enough arguments for format string
guquan_touzi form is Done!
开始进行企业index:

81afd8ebffd33dde1de1b2d8b41160b2 connect link is done!
上海浦旦石化工程有限公司企业无投资信息
上海浦旦石化工程有限公司企业投资数据写入失败
not enough arguments for format string
guquan_touzi form is Done!
开始进行企业index: 2282
Now count: 108
d25daaab77a4546f49cc012b50a4803d未公布财务数据
d25daaab77a4546f49cc012b50a4803d企业财务数据写入失败
'NoneType' object is not iterable
d25daaab77a4546f49cc012b50a4803d connect link is done!
上海业参石油化工设备有限公司企业无投资信息
上海业参石油化工设备有限公司企业投资数据写入失败
not enough arguments for format string
guquan_touzi form is Done!
开始进行企业index: 2283
Now count: 109
06d5ed647dc5b8e724d9c87592eccc0e未公布财务数据
06d5ed647dc5b8e724d9c87592eccc0e企业财务数据写入失败
'NoneType' object is not iterable
06d5ed647dc5b8e724d9c87592eccc0e connect link is done!
上海惠怡石化设备有限公司企业无投资信息
上海惠怡石化设备有限公司企业投资数据写入失败
not enough arguments for format string
guquan_touzi form is Done!
开始进行企业index: 2284
Now count: 110
396ecaa683b7e79f343a649f990cca05未公布财务数据
396ecaa683b7e79f343a649f990cca05企业财务数据写入失败
'NoneType' object is not iterable
396ecaa683b7e79f343a649f990cca05 connect link is done!

...

6ad3835c11b4e99df692efccd55a5dcc connect link is done!
中油万众石化(大连)有限公司企业无投资信息
中油万众石化(大连)有限公司企业投资数据写入失败
not enough arguments for format string
guquan_touzi form is Done!
开始进行企业index: 2685
Now count: 111
362bf20ea6e317225299e05cfb2bb642未公布财务数据
362bf20ea6e317225299e05cfb2bb642企业财务数据写入失败
'NoneType' object is not iterable
362bf20ea6e317225299e05cfb2bb642 connect link is done!
大连中油石化有限公司企业无投资信息
大连中油石化有限公司企业投资数据写入失败
not enough arguments for format string
guquan_touzi form is Done!
开始进行企业index: 2686
Now count: 112
c276aaab4bd1b52b7e1ad32aba39dd11未公布财务数据
c276aaab4bd1b52b7e1ad32aba39dd11企业财务数据写入失败
'NoneType' object is not iterable
c276aaab4bd1b52b7e1ad32aba39dd11 connect link is done!
盘锦益久石化有限公司企业无投资信息
盘锦益久石化有限公司企业投资数据写入失败
not enough arguments for format string
guquan_touzi form is Done!
开始进行企业index: 2687
Now count: 113
6063fcf4745dc8a1ce53a854cc1b2e99 financial info is done!
6063fcf4745dc8a1ce53a854cc1b2e99 connect link is done!
大连连重石化科技有限公司企业无投资信息
大连连重石化科技有限公司企业投资数据写入失败
not enough arguments for

c137dfeb370128b45dd5b1274c34c770 connect link is done!
大连瑞泉石化有限公司企业无投资信息
大连瑞泉石化有限公司企业投资数据写入失败
not enough arguments for format string
guquan_touzi form is Done!
开始进行企业index: 2712
Now count: 138
98ad4bdf0e0a72cc2e81180dc3dc7a5e未公布财务数据
98ad4bdf0e0a72cc2e81180dc3dc7a5e企业财务数据写入失败
'NoneType' object is not iterable
98ad4bdf0e0a72cc2e81180dc3dc7a5e connect link is done!
盘锦天逸顺石化商贸有限公司企业无投资信息
盘锦天逸顺石化商贸有限公司企业投资数据写入失败
not enough arguments for format string
guquan_touzi form is Done!
开始进行企业index: 2713
Now count: 139
6b9276da173ed759edb56e6075479a78未公布财务数据
6b9276da173ed759edb56e6075479a78企业财务数据写入失败
'NoneType' object is not iterable
6b9276da173ed759edb56e6075479a78 connect link is done!
营口万生隆石化有限公司企业无投资信息
营口万生隆石化有限公司企业投资数据写入失败
not enough arguments for format string
guquan_touzi form is Done!
开始进行企业index: 2714
Now count: 140
0bb19adc89de6ea48eb17150160eeb1c未公布财务数据
0bb19adc89de6ea48eb17150160eeb1c企业财务数据写入失败
'NoneType' object is not iterable
0bb19adc89de6ea48eb17150160eeb1c connect link is done!
盘锦鑫丰园石化

42f0bcc25f86a9642104d7fd37a89911 connect link is done!
中能石化(营口)有限公司企业无投资信息
中能石化(营口)有限公司企业投资数据写入失败
not enough arguments for format string
guquan_touzi form is Done!
开始进行企业index: 2740
Now count: 166
57dd61982b253d579a6f5a590021a3be未公布财务数据
57dd61982b253d579a6f5a590021a3be企业财务数据写入失败
'NoneType' object is not iterable
57dd61982b253d579a6f5a590021a3be connect link is done!
鞍山瑞达石化产品经销有限公司企业无投资信息
鞍山瑞达石化产品经销有限公司企业投资数据写入失败
not enough arguments for format string
guquan_touzi form is Done!
开始进行企业index: 2741
Now count: 167
dbe98fd006d76c7ea6004f8d6c5f579f未公布财务数据
dbe98fd006d76c7ea6004f8d6c5f579f企业财务数据写入失败
'NoneType' object is not iterable
dbe98fd006d76c7ea6004f8d6c5f579f connect link is done!
柳邕石化(大连)有限公司企业无投资信息
柳邕石化(大连)有限公司企业投资数据写入失败
not enough arguments for format string
guquan_touzi form is Done!
开始进行企业index: 2742
Now count: 168
8fcaa9709568a40ff00cdb7641e71c4a未公布财务数据
8fcaa9709568a40ff00cdb7641e71c4a企业财务数据写入失败
'NoneType' object is not iterable
8fcaa9709568a40ff00cdb7641e71c4a connect link is done!

大连德璞友合石化进出口有限公司企业投资数据写入失败
not enough arguments for format string
guquan_touzi form is Done!
开始进行企业index: 2766
Now count: 192
e5f53b91695158148570a85877f30f70未公布财务数据
e5f53b91695158148570a85877f30f70企业财务数据写入失败
'NoneType' object is not iterable
e5f53b91695158148570a85877f30f70 connect link is done!
辽宁亿辉石化工程建设有限公司企业无投资信息
辽宁亿辉石化工程建设有限公司企业投资数据写入失败
not enough arguments for format string
guquan_touzi form is Done!
开始进行企业index: 2767
Now count: 193
4afbf3f9cc16183c6439a0f5700a4d37未公布财务数据
4afbf3f9cc16183c6439a0f5700a4d37企业财务数据写入失败
'NoneType' object is not iterable
4afbf3f9cc16183c6439a0f5700a4d37 connect link is done!
迪泉子石化(大连)有限公司企业无投资信息
迪泉子石化(大连)有限公司企业投资数据写入失败
not enough arguments for format string
guquan_touzi form is Done!
开始进行企业index: 2768
Now count: 194
16be056fa18c10704e07f83d85658e52未公布财务数据
16be056fa18c10704e07f83d85658e52企业财务数据写入失败
'NoneType' object is not iterable
16be056fa18c10704e07f83d85658e52 connect link is done!
中油海路航石化(大连)有限公司企业无投资信息
中油海路航石化(大连)有限公司企业投资数据写入失败
not enough arguments

KeyboardInterrupt: 

In [49]:
def main(): #118911
    q = QichachaInfo()
    cookies_tuple = q.get_login()
    cookies = cookies_tuple[0]
    cookies_q = cookies_tuple[1]

    connection = get_dbservice_mysql_conn()
    sql = "SELECT `id`, `com` FROM `dic_company` WHERE `q_id` is null" #中国石化销售有限公司山东济南济阳第二十三加油站
    df = pandas.read_sql(sql,connection)
    for index in df.index:
        name = df.loc[index, 'com']
        print('开始进行%s企业,index是%s' %(name,index))
        logging.info('开始进行%s企业,index是%s' %(name,index))
        code = q.get_company_code(cookies, name)
        #print(code)
        s = StockMember(cookies)
        f = FinancialStatus(cookies)
        c = CompanyConnect(cookies)
        zl = ZhuanLi(cookies_q)
        cr = ChangeRecord(cookies_q)
        gm = GoldMerge(cookies_q)

        try:
            s.up_sql_guquan(name,code)
            s.up_sql_g_form(code)
        except Exception as e:
            print('%s无公司股权相关信息' %code)
            print(str(e))
        f.up_sql_financial(code)
        try:
            c.up_sql_c_form(code)
        except Exception as e:
            print('%s无公司企业关联信息' %code)
            print(str(e))
        try:
            zl.up_sql_zhuanli(code)
        except Exception as e:
            print('%s无公司专利相关信息' %code)
            print(str(e))
        try:
            cr.up_sql_change(code)
        except Exception as e:
            print('%s无公司变动/主要人员相关信息' %code)
            print(str(e))
        cr.update_status(code)
        try:
            gm.up_sql_merge(code)
        except Exception as e:
            print('%s无融资相关信息' %code)
            print(str(e))
        print('%s企业已完成' %name)
        time.sleep(random.uniform(4.4,17.7))

if __name__ == "__main__":
    main()

开始进行济南市历城区石油公司第一分公司企业,index是0
5575be6c2253825ba96a1c017ae4f0b7无公司股权相关信息
'NoneType' object is not subscriptable
5575be6c2253825ba96a1c017ae4f0b7未公布财务数据
5575be6c2253825ba96a1c017ae4f0b7企业财务数据写入失败
'NoneType' object is not iterable
5575be6c2253825ba96a1c017ae4f0b7 financial info is done!
5575be6c2253825ba96a1c017ae4f0b7 connect link is done!
5575be6c2253825ba96a1c017ae4f0b7无知识产权
5575be6c2253825ba96a1c017ae4f0b7 change info is done!
5575be6c2253825ba96a1c017ae4f0b7 partner info is done!
5575be6c2253825ba96a1c017ae4f0b7营业状态修改完成
5575be6c2253825ba96a1c017ae4f0b7无融资相关信息
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="financingList"]/table"}
  (Session info: headless chrome=71.0.3578.98)
  (Driver info: chromedriver=2.38.552522 (437e6fbedfa8762dec75e2c5b3ddb86763dc9dcb),platform=Windows NT 10.0.17134 x86_64)

济南市历城区石油公司第一分公司企业已完成
开始进行中国石化销售有限公司山东济南第九加油站企业,index是1
6e1469c9db68437dfb6bcf767ba87ce6无公司股权相关信息
'NoneType' object is not subscriptable
6e1469c9db

80f8528827d4584a984edc392eb965ec connect link is done!
80f8528827d4584a984edc392eb965ec无知识产权
80f8528827d4584a984edc392eb965ec change info is done!
80f8528827d4584a984edc392eb965ec partner info is done!
80f8528827d4584a984edc392eb965ec营业状态修改完成
80f8528827d4584a984edc392eb965ec无融资相关信息
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="financingList"]/table"}
  (Session info: headless chrome=71.0.3578.98)
  (Driver info: chromedriver=2.38.552522 (437e6fbedfa8762dec75e2c5b3ddb86763dc9dcb),platform=Windows NT 10.0.17134 x86_64)

中国石化销售有限公司山东济南商河第七加油站企业已完成
开始进行中国石化销售有限公司山东济南第五加油站企业,index是10
6a94cb7a62bd82fffa50703732e38132无公司股权相关信息
'NoneType' object is not subscriptable
6a94cb7a62bd82fffa50703732e38132未公布财务数据
6a94cb7a62bd82fffa50703732e38132企业财务数据写入失败
'NoneType' object is not iterable
6a94cb7a62bd82fffa50703732e38132 financial info is done!
6a94cb7a62bd82fffa50703732e38132 connect link is done!
6a94cb7a62bd82fffa50703732e38132无知识产权
6a94cb7a62bd82fffa507

dcebb255b46cb9618ecf3faae3f7de53营业状态修改完成
dcebb255b46cb9618ecf3faae3f7de53无融资相关信息
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="financingList"]/table"}
  (Session info: headless chrome=71.0.3578.98)
  (Driver info: chromedriver=2.38.552522 (437e6fbedfa8762dec75e2c5b3ddb86763dc9dcb),platform=Windows NT 10.0.17134 x86_64)

中国石化销售有限公司山东济南济阳第三十五加油站企业已完成
开始进行中国石化销售有限公司山东济南济阳第十一加油站企业,index是19
04cb8a2d15b50ac773269a8b1f798c8f无公司股权相关信息
'NoneType' object is not subscriptable
04cb8a2d15b50ac773269a8b1f798c8f未公布财务数据
04cb8a2d15b50ac773269a8b1f798c8f企业财务数据写入失败
'NoneType' object is not iterable
04cb8a2d15b50ac773269a8b1f798c8f financial info is done!
04cb8a2d15b50ac773269a8b1f798c8f connect link is done!
04cb8a2d15b50ac773269a8b1f798c8f无知识产权
04cb8a2d15b50ac773269a8b1f798c8f change info is done!
04cb8a2d15b50ac773269a8b1f798c8f partner info is done!
04cb8a2d15b50ac773269a8b1f798c8f营业状态修改完成
04cb8a2d15b50ac773269a8b1f798c8f无融资相关信息
Message: no such element: Un

开始进行中国石化销售有限公司山东济南章丘第三十七加油站企业,index是28
中国石化销售有限公司山东济南章丘第三十七加油站企业无投资信息
中国石化销售有限公司山东济南章丘第三十七加油站企业投资数据写入失败
not enough arguments for format string
中国石化销售有限公司山东济南章丘第三十七加油站 guquan info is done!
6ad971fb90dbcdb82ef5502306765308未公布财务数据
6ad971fb90dbcdb82ef5502306765308企业财务数据写入失败
'NoneType' object is not iterable
6ad971fb90dbcdb82ef5502306765308 financial info is done!
6ad971fb90dbcdb82ef5502306765308 connect link is done!
6ad971fb90dbcdb82ef5502306765308无知识产权
6ad971fb90dbcdb82ef5502306765308 change info is done!
6ad971fb90dbcdb82ef5502306765308 partner info is done!
6ad971fb90dbcdb82ef5502306765308营业状态修改完成
6ad971fb90dbcdb82ef5502306765308无融资相关信息
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="financingList"]/table"}
  (Session info: headless chrome=71.0.3578.98)
  (Driver info: chromedriver=2.38.552522 (437e6fbedfa8762dec75e2c5b3ddb86763dc9dcb),platform=Windows NT 10.0.17134 x86_64)

中国石化销售有限公司山东济南章丘第三十七加油站企业已完成
开始进行中国石化销售有限公司山东济南章丘第三十六加油站企业,index是29
中

c55afbe6df1f246a38ee1dec63458684未公布财务数据
c55afbe6df1f246a38ee1dec63458684企业财务数据写入失败
'NoneType' object is not iterable
c55afbe6df1f246a38ee1dec63458684 financial info is done!
c55afbe6df1f246a38ee1dec63458684 connect link is done!
c55afbe6df1f246a38ee1dec63458684无知识产权
c55afbe6df1f246a38ee1dec63458684 change info is done!
c55afbe6df1f246a38ee1dec63458684 partner info is done!
c55afbe6df1f246a38ee1dec63458684营业状态修改完成
c55afbe6df1f246a38ee1dec63458684无融资相关信息
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="financingList"]/table"}
  (Session info: headless chrome=71.0.3578.98)
  (Driver info: chromedriver=2.38.552522 (437e6fbedfa8762dec75e2c5b3ddb86763dc9dcb),platform=Windows NT 10.0.17134 x86_64)

中国石化销售有限公司山东济南章丘第十五加油站企业已完成
开始进行中国石化销售有限公司山东济南济阳第十五加油站企业,index是38
3b0c05035c64e1b04a4527cef3c1906c无公司股权相关信息
'NoneType' object is not subscriptable
3b0c05035c64e1b04a4527cef3c1906c未公布财务数据
3b0c05035c64e1b04a4527cef3c1906c企业财务数据写入失败
'NoneType' object is not it

7ca38a08d68ceb5c1f84c2273a0e2a41 connect link is done!
7ca38a08d68ceb5c1f84c2273a0e2a41无知识产权
7ca38a08d68ceb5c1f84c2273a0e2a41 change info is done!
7ca38a08d68ceb5c1f84c2273a0e2a41 partner info is done!
7ca38a08d68ceb5c1f84c2273a0e2a41营业状态修改完成
7ca38a08d68ceb5c1f84c2273a0e2a41无融资相关信息
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="financingList"]/table"}
  (Session info: headless chrome=71.0.3578.98)
  (Driver info: chromedriver=2.38.552522 (437e6fbedfa8762dec75e2c5b3ddb86763dc9dcb),platform=Windows NT 10.0.17134 x86_64)

中国石化销售有限公司山东济南章丘第四十二加油站企业已完成
开始进行中国石化销售有限公司山东济南章丘第三十五加油站企业,index是47
10660dfd5d713cff6cf6a3d8a0b247c4无公司股权相关信息
'NoneType' object is not subscriptable
10660dfd5d713cff6cf6a3d8a0b247c4未公布财务数据
10660dfd5d713cff6cf6a3d8a0b247c4企业财务数据写入失败
'NoneType' object is not iterable
10660dfd5d713cff6cf6a3d8a0b247c4 financial info is done!
10660dfd5d713cff6cf6a3d8a0b247c4 connect link is done!
10660dfd5d713cff6cf6a3d8a0b247c4无知识产权
10660dfd5d713cf

b9bc6c23ea9b8bb4b1aedacf2875dce1营业状态修改完成
b9bc6c23ea9b8bb4b1aedacf2875dce1无融资相关信息
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="financingList"]/table"}
  (Session info: headless chrome=71.0.3578.98)
  (Driver info: chromedriver=2.38.552522 (437e6fbedfa8762dec75e2c5b3ddb86763dc9dcb),platform=Windows NT 10.0.17134 x86_64)

中国石化销售有限公司山东济南章丘第三十三加油站企业已完成
开始进行中国石化销售有限公司山东济南章丘第四十八加油站企业,index是56
a1729867105fc0afd75cbc013cf371c6无公司股权相关信息
'NoneType' object is not subscriptable
a1729867105fc0afd75cbc013cf371c6未公布财务数据
a1729867105fc0afd75cbc013cf371c6企业财务数据写入失败
'NoneType' object is not iterable
a1729867105fc0afd75cbc013cf371c6 financial info is done!
a1729867105fc0afd75cbc013cf371c6 connect link is done!
a1729867105fc0afd75cbc013cf371c6无知识产权
a1729867105fc0afd75cbc013cf371c6 change info is done!
a1729867105fc0afd75cbc013cf371c6 partner info is done!
a1729867105fc0afd75cbc013cf371c6营业状态修改完成
a1729867105fc0afd75cbc013cf371c6无融资相关信息
Message: no such element: U

开始进行中国石化销售有限公司山东济南济阳第九加油站企业,index是65
75a24d8ecb495eb515abf17495416668无公司股权相关信息
'NoneType' object is not subscriptable
75a24d8ecb495eb515abf17495416668未公布财务数据
75a24d8ecb495eb515abf17495416668企业财务数据写入失败
'NoneType' object is not iterable
75a24d8ecb495eb515abf17495416668 financial info is done!
75a24d8ecb495eb515abf17495416668 connect link is done!
75a24d8ecb495eb515abf17495416668无知识产权
75a24d8ecb495eb515abf17495416668 change info is done!
75a24d8ecb495eb515abf17495416668 partner info is done!
75a24d8ecb495eb515abf17495416668营业状态修改完成
75a24d8ecb495eb515abf17495416668无融资相关信息
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="financingList"]/table"}
  (Session info: headless chrome=71.0.3578.98)
  (Driver info: chromedriver=2.38.552522 (437e6fbedfa8762dec75e2c5b3ddb86763dc9dcb),platform=Windows NT 10.0.17134 x86_64)

中国石化销售有限公司山东济南济阳第九加油站企业已完成
开始进行中国石化销售有限公司山东济南商河第一加油站企业,index是66
332859d07b46ae8569b3201bebb7e997无公司股权相关信息
'NoneType' object is not subscrip

6359be51d76f067e8d83cf7e7c90d958无知识产权
6359be51d76f067e8d83cf7e7c90d958 change info is done!
6359be51d76f067e8d83cf7e7c90d958 partner info is done!
6359be51d76f067e8d83cf7e7c90d958营业状态修改完成
6359be51d76f067e8d83cf7e7c90d958无融资相关信息
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="financingList"]/table"}
  (Session info: headless chrome=71.0.3578.98)
  (Driver info: chromedriver=2.38.552522 (437e6fbedfa8762dec75e2c5b3ddb86763dc9dcb),platform=Windows NT 10.0.17134 x86_64)

中国石化销售有限公司山东济南济阳第十三加油站企业已完成
开始进行中国石化销售有限公司山东济南长清第四加油站企业,index是75
3c4e2edbffe37c8a9c5bafd1bca2546b无公司股权相关信息
'NoneType' object is not subscriptable
3c4e2edbffe37c8a9c5bafd1bca2546b未公布财务数据
3c4e2edbffe37c8a9c5bafd1bca2546b企业财务数据写入失败
'NoneType' object is not iterable
3c4e2edbffe37c8a9c5bafd1bca2546b financial info is done!
3c4e2edbffe37c8a9c5bafd1bca2546b connect link is done!
3c4e2edbffe37c8a9c5bafd1bca2546b无知识产权
3c4e2edbffe37c8a9c5bafd1bca2546b change info is done!
3c4e2edbffe37c8a9c5

KeyboardInterrupt: 

In [48]:
q = QichachaInfo()
cookies_tuple = q.get_login()
cookies = cookies_tuple[0]
cookies_q = cookies_tuple[1]

cr = ChangeRecord(cookies_q)
#l = cr.get_change_data('f517fa63678872434e0b483bd082d250')
cr.up_sql_change('f517fa63678872434e0b483bd082d250')


f517fa63678872434e0b483bd082d250 change info is done!
f517fa63678872434e0b483bd082d250 partner info is done!


In [35]:
code = 'f517fa63678872434e0b483bd082d250'
url_t = 'https://www.qichacha.com/cbase_' + code
browser = webdriver.Chrome() 
browser.get(url_t) 

table = browser.find_element_by_xpath('//*[@id="Mainmember"]/table')
table_rows = table.find_elements_by_tag_name('tr')
table_list = []
for row in table_rows:
    table_list.append(row.find_elements_by_tag_name('td'))
decode_list =[]
for row in table_list:
    #for x in row:
        #print(x.text)
    cell_data = [cell.text.replace('\n','；') for cell in row]
    decode_list.append(cell_data)
decode_list.pop(0)
decode_list

1
申湘明
他关联 2 家公司 >
经理


[[], ['1', '申湘明；他关联 2 家公司 >', '经理']]

In [21]:
for i in decode_list:
    name_index =i[1].index('；')
    name = i[1][:name_index]
    i.pop(1)
    i.insert(1,name)
print(decode_list)

[['1', '于晓军', '董事长'], ['2', '刘海斌', '董事'], ['3', '秦捷', '董事'], ['4', '万雪松', '董事'], ['5', '吴志芬', '董事'], ['6', '张媛媛', '董事'], ['7', '赵辰清', '董事'], ['8', '于晓军', '总经理'], ['9', '倪越', '监事会主席'], ['10', '杨立', '监事'], ['11', '曾锋', '监事']]


In [63]:
l= [code,]+['']*3
l

['01ea0f31cdf66024e8a7d26a26190547', '', '', '']

In [3]:
from selenium import webdriver

class TestB(object):
    
    browser = webdriver.Chrome()
    
    def get_b(self):
        url = 'https://www.qichacha.com/firm_6bc4a7dd26e659c333195c1f486e86f2.shtml#run'
        self.browser.get(url)
        
t = TestB()
t.get_b()